In [2]:
import random
import string
import sqlite3
from datetime import datetime, timedelta

# Function to generate a random username
def generate_username():
    username = ''.join(random.choice(string.ascii_lowercase) for _ in range(8))
    return username

# Function to generate a random password
def generate_password():
    password = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(8))
    return password

# Function to generate a random full name
def generate_fullname():
    first_names = ["Christopher", "Hugh", "Michael", "Henry", "Taron", "Pedro", "Bruce", "Bruce", "Jensen", "Arthur"]
    last_names = ["Nolan", "Jackman", "Grzesiek", "Cavill", "Egerton", "Pascal", "Wayne", "Mathers", "Ackles", "Morgan"]
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    fullname = f"{first_name} {last_name} "
    return fullname

# Function to generate a random address
def generate_address():
    address = f"{random.randint(1, 999)} {random.choice(string.ascii_uppercase)} St, {random.choice(['Lubbock', 'Dallas', 'Austin'])}"
    return address

# Function to generate a random phone number
def generate_phonenumber():
    phonenumber = ''.join(random.choice(string.digits) for _ in range(10))
    return phonenumber

# Function to generate a random email
def generate_email(fullname):
    email = f"{fullname.replace(' ', '').lower()}@gmail.com"
    return email

# Connect to the SQLite database
db = sqlite3.connect('store.db')
cursor = db.cursor()

# Generate 1000 users
for _ in range(1000):
    username = generate_username()
    password = generate_password()
    fullname = generate_fullname()
    address = generate_address()
    phonenumber = generate_phonenumber()
    email = generate_email(fullname)

    # Insert user data into the Users table
    cursor.execute("INSERT INTO Users (username, password, fullname, address, phonenumber, email) VALUES (?, ?, ?, ?, ?, ?)",
                   (username, password, fullname, address, phonenumber, email))

# Generate 2000 products
for _ in range(2000):
    userid = random.randint(1, 1000)
    name = f"{random.choice(['Apple', 'Samsung', 'Sony'])} {random.choice(['Phone', 'Tablet', 'Laptop', 'Headphones'])}"
    description = f"High-quality {name}"
    price = round(random.uniform(10, 1000), 2)
    quantity = random.randint(1, 100)

    # Insert product data into the Products table
    cursor.execute("INSERT INTO Products (userid, name, description, price, quantity) VALUES (?, ?, ?, ?, ?)",
                   (userid, name, description, price, quantity))

# Generate 10,000 orders
orderidcounter = 1
start_date = datetime.strptime('2023-07-01', '%Y-%m-%d')
end_date = datetime.strptime('2023-07-31', '%Y-%m-%d')
for _ in range(10000):
    userid = random.randint(1, 1000)
    date = (start_date + timedelta(days=random.randint(0, (end_date - start_date).days))).strftime("%Y-%m-%d")

    # Generate random number of products (up to 10) for each order
    num_products = random.randint(1, 10)
    totalcost = 0.0  # Initialize the totalcost variable

    # Insert order data into the Orders table
    cursor.execute("INSERT INTO Orders (userid, date, totalcost) VALUES (?, ?, ?)",
                   (userid, date, totalcost))
    orderid = cursor.lastrowid

    for _ in range(num_products):
        productid = random.randint(1, 2000)
        product_quantity = random.randint(1, 5)

        # Get the product price
        cursor.execute("SELECT price FROM Products WHERE productid = ?", (productid,))
        price = cursor.fetchone()[0]

        # Calculate the cost for the order item
        cost = price * product_quantity
        totalcost += cost

        # Insert order item data into the OrderItems table
        cursor.execute("INSERT INTO OrderItems (orderid, productid, quantity, userid, cost) VALUES (?, ?, ?, ?, ?)",
                       (orderid, productid, product_quantity, userid, cost))

    # Update the total cost for the order in the Orders table
    cursor.execute("UPDATE Orders SET totalcost = (SELECT SUM(cost) FROM OrderItems WHERE orderid = ?) WHERE orderid = ?",
                   (orderid, orderid))

    orderidcounter += 1

# Commit the changes to the database
db.commit()

# Close the database connection
db.close()
